# Overview

> The goal of this notebook is to show how we could create a list of Medication* resources mapped to a single medication resource type.

Specifically, we'll create a `Bundle` after mapping `MedicationAdministration`, `MedicationDispense` and `MedicationRequest` to `MedicationStatement` - but we could use the same approach to map to any target type.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pete88b/smart-on-fhir-client-py-demo/blob/main/vulcan_medication_bundle_getting_started.ipynb)

## Quick Links

- https://www.hl7.org/fhir/bundle.html
- https://www.hl7.org/fhir/list.html

## Next steps

Might we want to
- create one bundle per subject
- define some kind of order of entries in the bundle
- think about how we handle resources that fail validation 

## Why are we not using `List`

We need a container that allows us to include complete resources.

The following JSON shows how a MedicationStatement can be part of a bundle entry;

```
{'entry': [
  {'resource': 
    {'id': '1733210',
     'subject': {'reference': 'Patient/1732972'},
     'resourceType': 'MedicationStatement'}}],
 'resourceType': 'Bundle'}
```

but `List` entries must be references;

```
{'entry': [
   {'item': {'reference': 'MedicationStatement/1733210'}}],
 'resourceType': 'List'}
```

and we won't have valid references for mapped/transformed resources (i.e. for MedicationDispense mapped to MedicationStatement)

In [ ]:
IN_COLAB = 'google.colab' in str(get_ipython())
if IN_COLAB:
    !pip install -Uqq git+https://github.com/smart-on-fhir/client-py.git

In [ ]:
import IPython, json, requests
from datetime import datetime, timezone
from collections import Counter
from pathlib import Path
from fhirclient import client
from fhirclient.models.annotation import Annotation
from fhirclient.models.bundle import Bundle, BundleEntry
from fhirclient.models.dosage import Dosage
from fhirclient.models.fhirreference import FHIRReference   
from fhirclient.models.humanname import HumanName
from fhirclient.models.medication import Medication
from fhirclient.models.medicationadministration import MedicationAdministration
from fhirclient.models.medicationdispense import MedicationDispense
from fhirclient.models.medicationrequest import MedicationRequest
from fhirclient.models.medicationstatement import MedicationStatement
from fhirclient.models.patient import Patient
from fhirclient.models.fhirdate import FHIRDate
from fhirclient.models.list import List, ListEntry

In [ ]:
settings = {
    'app_id': 'my_web_app',
    'api_base': 'http://hapi.fhir.org/baseR4'
}
smart = client.FHIRClient(settings=settings)

### Why are we reading FHIR resources as raw JSON?

Lots of the data in the test servers fails validation, making it really hard to find examples that work with the FHIR py classes.

```
search = MedicationRequest.where(struct={})
print(search.construct())
resources = search.perform_resources(smart.server)
```

the `perform_resources` call &uarr; fails with validation errors &darr; and we can't access the data )o:

```
FHIRValidationError: {root}:
  entry.1:
    resource:
      'Non-optional property "subject" on <fhirclient.models.medicationrequest.MedicationRequest object at 0x00000281E4249790> is missing'
      'Non-optional property "status" on <fhirclient.models.medicationrequest.MedicationRequest object at 0x00000281E4249790> is missing'
      'Non-optional property "medication" on <fhirclient.models.medicationrequest.MedicationRequest object at 0x00000281E4249790> is missing'
      'Non-optional property "intent" on <fhirclient.models.medicationrequest.MedicationRequest object at 0x00000281E4249790> is missing'
```

In [ ]:
def get_bundle_as_raw_json(api_base, resource_type, url_suffix=None):
    "GET a bundle of resources of a specific type"
    url=f'{api_base}/{resource_type}'
    if url_suffix is not None:
        url+=url_suffix
    print('GET',url)
    return requests.get(url).json()

def get_next_bundle_as_raw_json(json_response):
    "GET the next set of results"
    if len(json_response['link']) == 0: return None
    url = json_response['link'][1]['url']
    print('GET',url)
    return requests.get(url).json()

### save some bundles to file

this will make repeatable runs of this notebook possible if resources on the http://hapi.fhir.org/baseR4 server change

In [ ]:
def resource_bundle_to_file(api_base, resource_type, url_suffix=None):
    json_response = get_bundle_as_raw_json(api_base, resource_type, url_suffix)
    file_number = 0
    with open(f'data/{resource_type}_bundle_{file_number}.json','w') as f:
        json.dump(json_response, f, indent=2)
    # follow "next" links - this might makes lots of requests depending on how much data the server has
#     json_response = get_next_bundle_as_raw_json(json_response)
#     while json_response is not None:
#         file_number += 1
#         with open(f'data/{resource_type}_bundle_{file_count}.json','w') as f:
#             json.dump(json_response, f, indent=2)

In [ ]:
if not IN_COLAB: # remove this line if you want to save files in colab
    Path('data').mkdir(exist_ok=True)
    for resource_type, url_suffix in [
            ['Patient','?name=RWD-Vulcan'],
            ['MedicationRequest',None],
            ['MedicationDispense',None],
            ['MedicationAdministration',None],
            ['MedicationStatement',None]]:
        resource_bundle_to_file(settings['api_base'], resource_type, url_suffix)

GET http://hapi.fhir.org/baseR4/Patient?name=RWD-Vulcan
GET http://hapi.fhir.org/baseR4/MedicationRequest
GET http://hapi.fhir.org/baseR4/MedicationDispense
GET http://hapi.fhir.org/baseR4/MedicationAdministration
GET http://hapi.fhir.org/baseR4/MedicationStatement


In [ ]:
def load_from_json(resource_type, file_number=0):    
    with open(f'data/{resource_type}_bundle_{file_number}.json') as f:
        return json.load(f)
    
def load_bundle(resource_type, file_number=0):    
    return Bundle(load_from_json(resource_type, file_number))

def convert_json_bundle_to_list_of_resources(json_bundle, resource_type):
    result = []
    for entry in json_bundle['entry']:
        result.append(resource_type(entry), False)
    return result

The following cells show how the function above work and/or fail with validation errors

In [ ]:
# convert_json_bundle_to_list_of_resources(load_from_json('MedicationRequest'), MedicationRequest)

In [ ]:
# load_from_json('MedicationRequest') # return py dict

In [ ]:
# load_bundle('MedicationRequest').entry[0].resource.as_json() # probably fails validation

### Can we use the "RWD-Vulcan" data?

It would be great if we had examples of patients with all kinds of Medication* resources but ...

it looks like we have patients with MedicationStatement &darr; (i.e. no other Medication* resources)

In [ ]:
search=Patient.where({'name': 'RWD-Vulcan'})
search = search.include('subject', MedicationStatement, reverse=True)
# search = search.include('subject', MedicationRequest, reverse=True) # -> Counter({'Patient': 10})
list_of_resources = search.perform_resources(smart.server)
Counter([r.__class__.__name__ for r in list_of_resources])

Counter({'Patient': 10, 'MedicationStatement': 179})

### Should we create FHIR bundles with the python client?

&darr; this is problematic as lots of resources fail validation ...

In [ ]:
bundle = Bundle()
# not sure it makes sense to populate id or identifier as we're just using list as a container
bundle.type = 'collection'
bundle.timestamp = FHIRDate(datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ'))
bundle.entry = []
# lots of MedicationStatement resources pass validation
bundle.entry.append(BundleEntry())
bundle.entry[-1].resource = MedicationStatement.read('1733210', smart.server)
# but ... most MedicationRequest resources fail
# for resource in MedicationRequest.where({}).perform_resources(smart.server):
#     bundle.entry.append(BundleEntry())
#     bundle.entry[-1].resource = resource
bundle.as_json()

{'entry': [{'resource': {'id': '1733210',
    'meta': {'lastUpdated': '2021-01-11T21:09:19.051+00:00',
     'source': '#MYi0ZdRYytOrGkxj',
     'versionId': '1'},
    'dosage': [{'doseAndRate': [{'doseQuantity': {'value': 90.0}}]}],
    'effectivePeriod': {'start': '2010-08-29T00:00:00+00:00'},
    'identifier': [{'value': 'rwd-MedicationStatement-5'}],
    'medicationCodeableConcept': {'coding': [{'code': '197392',
       'display': 'Baclofen 20 MG Oral Tablet',
       'system': 'http://www.nlm.nih.gov/research/umls/rxnorm'}]},
    'status': 'active',
    'subject': {'reference': 'Patient/1732972'},
    'resourceType': 'MedicationStatement'}}],
 'timestamp': '2021-08-05T19:18:51Z',
 'type': 'collection',
 'resourceType': 'Bundle'}

## What would it look like if we work with py dictionaries instead ...

if our taget type is the same as the source type, we could copy the entry list into the target ... but we might want to remove `fullUrl` and just keep resources

In [ ]:
bundle = dict(resourceType='Bundle', 
              type='collection', 
              timestamp=datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ'), 
              entry=[])
bundle['entry'].extend(get_bundle_as_raw_json(settings['api_base'],'MedicationRequest')['entry'])

GET http://hapi.fhir.org/baseR4/MedicationRequest


uncomment and run the following cell if you want to see the bundle we just created

In [ ]:
# bundle

### Helper functions to map between different resource types

In [ ]:
def pull_attr(resouce,attr_path):
    "Pull a value from `resource` if we can find the attribute specified"
    for _attr_path in attr_path.split(' OR '):
        r,found=resouce,True
        for _attr in _attr_path.split('.'):
            if _attr not in r: 
                found=False; break
            r=r[_attr]
        if found: return r

In [ ]:
def transform(resource,mapping):
    "Pull data from `resource` to create a new instance using `mapping`"
    result={}
    for k in mapping:
        attr=pull_attr(resource,mapping[k])
        if attr is not None: result[k]=attr
    return result

## Map medication request to medication statement

The keys of `medication_request_to_medication_statement` are medication statement attributes, its values are medication request attributes.

In [ ]:
medication_request_to_medication_statement=dict(
    id='id', meta='meta', implicitRules='implicitRules', language='language', text='text', contained='contained', extension='extension', modifierExtension='modifierExtension',
    identifier='identifier',
#     basedOn='id', #TODO: make this a reference
    partOf='partOf',
    status='status',
    statusReason='statusReason',
    category='category',
    medicationCodeableConcept='medicationCodeableConcept',
    medicationReference='medicationReference',
    subject='subject',
    context='encounter',
#     effectiveDateTime                         # might be better to leave this blank as we have dosage
    effectivePeriod='dosageInstruction.timing', # might be better to leave this blank as we have dosage
    dateAsserted='authoredOn',
    informationSource='requester',
#     derivedFrom='id', #TODO: make this a reference
    reasonCode='reasonCode',
    reasonReference='reasonReference',
    note='note',
    dosage='dosageInstruction'
)

In [ ]:
def transform_medication_request_to_medication_statement(resource):
    result=transform(resource, medication_request_to_medication_statement)
    # TODO: use absolute URL
    # if we had a bundle entry (not jsut a medication request) we could use fullUrl
    result['basedOn']=result['derivedFrom']=[dict(reference=f'MedicationRequest/{resource["id"]}')]
    result['resourceType']='MedicationStatement'
    return result

Read a MedicationRequest that we can test with &darr;

In [ ]:
# we can pull a request out of a bundle ...
# medication_request = get_bundle_as_raw_json(settings['api_base'],'MedicationRequest')['entry'][1]['resource']
# ... or read a request by ID
medication_request = MedicationRequest.read('1465875', smart.server).as_json()

### Show how `transform` works

We can create a `MedicationStatement` like instance from a `MedicationRequest` by copying fields from request to statement using `medication_request_to_medication_statement` 

Note: the result of this basic transform will be missing `derivedFrom` etc

In [ ]:
tfm = transform(medication_request, medication_request_to_medication_statement)
tfm['subject']

{'display': 'Peter James Chalmers', 'reference': 'Patient/1293520'}

To populate all fields, we need to transform using `transform_medication_request_to_medication_statement`

In [ ]:
tfm = transform_medication_request_to_medication_statement(medication_request)
tfm['subject']

{'display': 'Peter James Chalmers', 'reference': 'Patient/1293520'}

We should be able to use this dictionary to create a MedicationStatement that passes validation.

Note: Calling `as_json` on the `MedicationStatement` causes validation to be run

In [ ]:
str(MedicationStatement(tfm).as_json())[:100]

"{'id': '1465875', 'meta': {'lastUpdated': '2021-08-04T13:54:11.507+00:00', 'source': '#TuNVIftMM8wZB"

If you'd like to see the full JSON of the test medication request of the medication statement that we created, uncomment and run the following cells.

Note: `tfm` and `MedicationStatement(tfm).as_json()` should show us the same content - but fields might be ordered differently.

In [ ]:
# medication_request

In [ ]:
# tfm
# MedicationStatement(tfm).as_json()

## Convert a bundle of MedicationRequest to a bundle of MedicationStatement

we need a few more helper functions &darr;

In [ ]:
def new_bundle():
    return dict(resourceType='Bundle', 
                type='collection', 
                timestamp=datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ'), 
                entry=[])

def add_entry(bundle, resource, full_url=None):
    entry = dict(resource=resource)
    if full_url is not None: entry['fullUrl'] = full_url
    bundle['entry'].append(entry)

In [ ]:
def pull_resources_from_bundle(bundle):
    return [entry['resource'] for entry in bundle['entry']]

and now we can create a bundle of `MedicationStatement`s from a bundle of `MedicationRequest`s

In [ ]:
bundle = new_bundle()
resources = pull_resources_from_bundle(get_bundle_as_raw_json(settings['api_base'], 'MedicationRequest'))
for r in [transform_medication_request_to_medication_statement(r) for r in resources]:
    add_entry(bundle, r)

GET http://hapi.fhir.org/baseR4/MedicationRequest


uncomment and run the following cell if you want to see the bundle we just created

In [ ]:
# bundle

## Map medication dispense to medication statement

In [ ]:
medication_dispense_to_medication_statement=dict(
    id='id', meta='meta', implicitRules='implicitRules', language='language', text='text', contained='contained', extension='extension', modifierExtension='modifierExtension',
    identifier='identifier',
    basedOn='authorizingPrescription',
    partOf='partOf',
    status='status',
    statusReason='statusReason',
    category='category',
    medicationCodeableConcept='medicationCodeableConcept',
    medicationReference='medicationReference',
    subject='subject',
    context='context',
#     effectiveDateTime                         # might be better to leave this blank as we have dosage
    effectivePeriod='dosageInstruction.timing', # might be better to leave this blank as we have dosage
    #     dateAsserted # pull from event history?
    informationSource='performer',
#     derivedFrom='id', #TODO: make this a reference
#     reasonCode
#     reasonReference
    note='note',
    dosage='dosageInstruction'
)

In [ ]:
def transform_medication_dispense_to_medication_statement(resource):
    result=transform(resource, medication_dispense_to_medication_statement)
    # TODO: use absolute URL
    # if we had a bundle entry (not jsut a medication request) we could use fullUrl
    result['derivedFrom']=[dict(reference=f'MedicationDispense/{resource["id"]}')]
    if 'basedOn' in result and not isinstance(result['basedOn'],list):
        result['basedOn']=[result['basedOn']]
    result['resourceType']='MedicationStatement'
    return result

## Map medication administration dosage to dosage

In [ ]:
medication_administration_dose_to_dose=dict(
    text='text',
    site='site',
    route='route',
    method='method',
    doseQuantity='dose',
    rateRatio='rateRatio',
    rateQuantity='rateQuantity'
)

## Map medication administration to medication statement

In [ ]:
medication_administration_to_medication_statement=dict(
    id='id', meta='meta', implicitRules='implicitRules', language='language', text='text', contained='contained', extension='extension', modifierExtension='modifierExtension',
    identifier='identifier',
    basedOn='request',
    partOf='partOf',
    status='status',
    statusReason='statusReason',
    category='category',
    medicationCodeableConcept='medicationCodeableConcept',
    medicationReference='medicationReference',
    subject='subject',
    context='context',
    effectiveDateTime='effectiveDateTime',
    effectivePeriod='effectivePeriod',
#     dateAsserted # pull from event history?
    informationSource='performer',
#     derivedFrom='id', #TODO: make this a reference
#     reasonCode
    reasonReference='reasonReference',
    note='note',
#     dosage='dosage' # need to map MedicationAdministrationDosage to Dosage
)

In [ ]:
def transform_medication_administration_to_medication_statement(resource):
    result = transform(resource, medication_administration_to_medication_statement)
    if 'dosage' in result:
        result['dosage'] = [transform(result['dosage'], medication_administration_dose_to_dose)]
    # TODO: use absolute URL
    # if we had a bundle entry (not jsut a medication request) we could use fullUrl
    result['derivedFrom']=[dict(reference=f'MedicationAdministration/{resource["id"]}')]
    if 'basedOn' in result and not isinstance(result['basedOn'],list):
        result['basedOn']=[result['basedOn']]
    result['resourceType']='MedicationStatement'
    return result

put all of the transform functions into a dictionary &darr; to make the simpify creation of the next bundle

In [ ]:
to_medication_statement_functions = dict(
    MedicationAdministration=transform_medication_administration_to_medication_statement,
    MedicationDispense=transform_medication_dispense_to_medication_statement,
    MedicationRequest=transform_medication_request_to_medication_statement,
    MedicationStatement=lambda r: r # do nothing for statement
)

# Create a `Bundle` after mapping `MedicationAdministration`, `MedicationDispense` and `MedicationRequest` to `MedicationStatement`

The following cell 
- reads all 4 Medication* resources,
- converts them to `MedicationStatement` and
- puts them all in the same bundle

In [ ]:
bundle = new_bundle()
for resource_type in to_medication_statement_functions: 
    resources = pull_resources_from_bundle(get_bundle_as_raw_json(settings['api_base'], resource_type))
    for r in [to_medication_statement_functions[resource_type](r) for r in resources]:
        add_entry(bundle, r)

GET http://hapi.fhir.org/baseR4/MedicationAdministration
GET http://hapi.fhir.org/baseR4/MedicationDispense
GET http://hapi.fhir.org/baseR4/MedicationRequest
GET http://hapi.fhir.org/baseR4/MedicationStatement


uncomment and run the following cell if you want to see the bundle we just created

In [ ]:
# bundle

## Have we made a valid bundle?

We can use the python client to validate the bundle we just created with: `Bundle(bundle)` but ...

we have entries with missing mandatory fields (like mesication* and subject) - the next cell removes these invalid resources from the bundle

In [ ]:
bundle['entry']=[e for e in bundle['entry'] if 
                 'medicationCodeableConcept' in e['resource'] or
                 'medicationReference' in e['resource']]

bundle['entry']=[e for e in bundle['entry'] if 
                 'subject' in e['resource']]

In [ ]:
# we can use this kind of code to find and look at invalid resources
# for i, e in enumerate(bundle['entry']):
#     r = e['resource']
#     if 'medicationCodeableConcept' not in r and 'medicationReference' not in r:
#         print(i,r)

In [ ]:
# now that the invalid resources are removed, we should see just a few warnings
Bundle(bundle, False)

entry.10:
  resource:
    informationSource.0:
      Superfluous entry "function" in data for <fhirclient.models.fhirreference.FHIRReference object at 0x00000131C4B15A00>
      Superfluous entry "actor" in data for <fhirclient.models.fhirreference.FHIRReference object at 0x00000131C4B15A00>


In [ ]:
# here's the informationSource part of the resource that caused the warnings above
bundle['entry'][10]['resource']['informationSource']

[{'function': {'coding': [{'system': 'http://www.nlm.nih.gov/research/umls/rxnorm',
     'code': '285018',
     'display': 'Lantus 100 unit/ml injectable solution'}]},
  'actor': {'reference': 'RelatedPerson/248511'}}]